In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Glioblastoma/GSE249289'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiles of glioblastoma tumorspheres cultured in diverse platforms"
!Series_summary	"We studied five patients with IDH1 wild-type glioblastoma who were newly diagnosed with no treatment history via surgery, chemotherapy, or radiotherapy. Patient-derived glioblastoma tumorspheres (TSs) were established from fresh tissue specimens, and they were cultured in divserse platforms."
!Series_overall_design	"Gene expression profiles of five glioblastoma tumorspheres cultured in diverse platforms (collagen, normal ECM, tumor ECM, and mouse xenograft)"
Sample Characteristics Dictionary:
{0: ['tissue: Brain'], 1: ['Sex: Male', 'Sex: Female'], 2: ['age: 61', 'age: 56', 'age: 57', 'age: 67'], 3: ['tumorsphere: TS13-20', 'tumorsphere: TS13-64', 'tumorsphere: TS14-08', 'tumorsphere: TS14-15', 'tumorsphere: TS15-88'], 4: ['culture platform: Collagen', 'culture platform: nECM', 'culture platform: tECM', 'culture platform: mouse xenograft']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Gene Expression Data Availability
is_gene_available = True

# Step 2: Variable Availability and Data Type Conversion

# Data Availability
trait_row = 0  # assuming the presence of 'tissue: Brain' is indicative of Glioblastoma
age_row = 2    # age data is clearly indicated
gender_row = 1 # gender data is available with Male and Female records

# Data Type Conversion Functions

# Convert trait ('Glioblastoma') data
def convert_trait(value):
    # Trait is binary, presence ('tissue: Brain') means 1 (has Glioblastoma)
    if 'tissue: Brain' in value:
        return 1
    return None

# Convert age data
def convert_age(value):
    try:
        return int(value.split(": ")[1])
    except ValueError:
        return None

# Convert gender data
def convert_gender(value):
    gender = value.split(": ")[1]
    if gender.lower() == 'male':
        return 1
    elif gender.lower() == 'female':
        return 0
    return None

# Save Metadata
save_cohort_info('GSE249289', './preprocessed/Glioblastoma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Glioblastoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Glioblastoma/trait_data/GSE249289.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM7933102': [1, 61, 1], 'GSM7933103': [1, 61, 1], 'GSM7933104': [1, 61, 1], 'GSM7933105': [1, 61, 1], 'GSM7933106': [1, 61, 1], 'GSM7933107': [1, 61, 1], 'GSM7933108': [1, 61, 1], 'GSM7933109': [1, 61, 1], 'GSM7933110': [1, 61, 1], 'GSM7933111': [1, 61, 1], 'GSM7933112': [1, 61, 1], 'GSM7933113': [1, 61, 1], 'GSM7933114': [1, 61, 1], 'GSM7933115': [1, 61, 1], 'GSM7933116': [1, 61, 1], 'GSM7933117': [1, 61, 1], 'GSM7933118': [1, 61, 1], 'GSM7933119': [1, 61, 1], 'GSM7933120': [1, 56, 0], 'GSM7933121': [1, 56, 0], 'GSM7933122': [1, 56, 0], 'GSM7933123': [1, 56, 0], 'GSM7933124': [1, 56, 0], 'GSM7933125': [1, 57, 0], 'GSM7933126': [1, 57, 0], 'GSM7933127': [1, 57, 0], 'GSM7933128': [1, 57, 0], 'GSM7933129': [1, 57, 0], 'GSM7933130': [1, 57, 0], 'GSM7933131': [1, 57, 0], 'GSM7933132': [1, 57, 0], 'GSM7933133': [1, 67, 1], 'GSM7933134': [1, 67, 1], 'GSM7933135': [1, 67, 1], 'GSM7933136': [1, 67, 1], 'GSM7933137': [1, 67, 1], 'GSM7933138': [1, 67, 1], 'GSM7933139': [1, 67, 1], 'GSM7933140

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify keys for identifier and gene symbol
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Glioblastoma/gene_data/GSE249289.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Glioblastoma')

# 4. Save the cohort information.
save_cohort_info('GSE249289', './preprocessed/Glioblastoma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Glioblastoma/GSE249289.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Glioblastoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Glioblastoma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 57.0
  50% (Median): 61.0
  75%: 61.0
Min: 56.0
Max: 67.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 13 occurrences. This represents 27.66% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

